In [1]:
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.losses import MeanAbsolutePercentageError as MAPE
from random import choice,random
import tensorflow as tf
from pprint import pformat,pprint
import keras
import pickle
from keras import layers, models, regularizers, backend, utils
import keras.backend.tensorflow_backend as tfback
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge
from IPython.core.display import display, HTML
from tqdm.keras import TqdmCallback

#widen the container width if you want
#display(HTML("<style>.container { width:90% !important; }</style>"))

K = keras.backend
mape = MAPE()
%matplotlib inline

Using TensorFlow backend.


Check if the nvidia-docker container supplied GPU and the tf is using it:

In [2]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Read in the select data Data. This splits the data up into inputs and outputs, and normalizes both.

In [3]:
files = "Sydney Tasmania Perth Adelaide".split()

xs_tests = dict()
ys_tests = dict()
xs_trains = dict()
ys_trains = dict()
for file in files:
    csvfile = open(f"/notebooks/WECs_DataSet/{file}_Data.csv")    
    reader = csv.reader(csvfile, delimiter = ',')
    rows = [[float(v) for v in row] for row in reader]
    rows = np.asarray(rows, dtype='float32')
    xs = rows[:,0:32]
    ys = rows[:,32:49]
    ys = ys / np.max(ys)
    xs = xs / np.max(xs)
    xs_train, xs_test, ys_train, ys_test = train_test_split(xs,ys,test_size = 0.2,random_state=42)
    xs_tests[file] = xs_test
    ys_tests[file] = ys_test
    xs_trains[file] = xs_train
    ys_trains[file] = ys_train

Given a model, this function gets the MAPE(Mean Absolute Percentage Error) of the model using test data.

In [4]:
def get_error(file,model):
    ys_predictions = model.predict(xs_tests[file])
    mape = tf.keras.losses.MeanAbsolutePercentageError()
    error = mape(ys_predictions, ys_tests[file]).numpy()
    return error

In [5]:
def train_model(file,patience=100,verbose=0):
    model = models.Sequential()
    model.add(layers.Dense(1024, input_dim=32, activation = "relu"))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(2048, activation = "relu"))
    model.add(layers.Dense(1024, activation = "relu"))
    model.add(layers.Dense(17, activation = 'linear'))
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience)
    model.compile(keras.optimizers.Adam(learning_rate=0.0003),loss="mean_absolute_percentage_error")
    loss_hist = model.fit(xs_trains[file], ys_trains[file], epochs = 15000, shuffle=True,
                          verbose=0, validation_data = (xs_tests[file], ys_tests[file]), 
                          batch_size=2048, callbacks=[callback,TqdmCallback(verbose=verbose)])
    return model,loss_hist

Save the model and the history

In [6]:
for file in files:
    print(file)
    model,loss_hist = train_model(file,patience=100,verbose=1)
    model.save(f"models/{file}_model.h5")
    with open(f"models/{file}_hist.pickle","wb") as f:
        pickle.dump(loss_hist,f)
    linear_model = LinearRegression().fit(xs_trains[file],ys_trains[file])

    print(f"Linear Model MAPE:\t {get_error(file,linear_model)}")
    print(f"Sequential Model MAPE:\t {get_error(file,model)}")
    print("\n"*5)

Sydney



Linear Model MAPE:	 3.2048799991607666
Sequential Model MAPE:	 1.2961041927337646






Tasmania



Linear Model MAPE:	 6.566184043884277
Sequential Model MAPE:	 5.396560192108154






Perth



Linear Model MAPE:	 6.818752288818359
Sequential Model MAPE:	 5.166650295257568






Adelaide



Linear Model MAPE:	 6.791271686553955
Sequential Model MAPE:	 4.96780252456665






